# Task
Вытаскиваю все возможные признаки из временной метки

В качестве таргета беру ситуацию когда прирос на следующем периоде > 1%, а нижняя тень < 0.25%.

Я уже проверил, что в выброке всего 5% таких событий.

Идея делать классификацию не по закрытию бара, а по максимуму.
Мне кажется случаев когда нижняя тень < 0.25%, а верхняя > 1% больше чем с закрытием.
Это завтра посчитать в ноутбуке рисерча.

А отношение награда/риск это как бы мой гипераметр, который можно подбирать. Вместе с нижней тенью. Мне же по сути не важно что ловить, как размечать данные.

А основная суть такой разметки данных - это отбор событий когда в течении часа или любого другого периода потенциально положительное движение идет с существенным перекосом вероятности, как 4к1.


МО = 4 * 0.5 - 1 * 0.5 = 2 - 0.5 = 1.5 риска на сделку получается матожидание при угадайке алгоритма всего 50%

И точка безубытка когда МО = 0, где x - угадайка алгоритма
 
0 = 4 * x - 1 * (1 - x)

4х -1 - х = 0
3х = 1
Х = 33.33%

Т.е. При угадайке в 33.33% я буду в нуле при нулевых издержках.

Для соотношении доходность/риск как 5к1
5х - 1 - х = 0
Х = 25%



Т.е. Это что-то типа детекции спама или скорее определение редкой болезни. Только в моем случае ложно-положительные исходы нужно сводить к минимуму, пусть я не угадаю все исходы, но главное минимизировать ложно-положительные т.к. по ним я совершаю сделки и буду получать убытки.

И получается что для меня опастность представляют ложно положительные результаты, на них я теряю деньги. Мне лучше постоять в стороне.

Тут важно правильно подобрать метрику. 


# Import modules

In [1]:
# для автозагрузки изменений в модулях без необходимости перезагрузки kelner
%load_ext autoreload
%autoreload 2

# необходимо для корректного импорта своих модулей в JupyterLab
import sys
current_dir = sys.path[0]
project_dir = 'SF_DS_Pro'
last_position = current_dir.find(project_dir) + len(project_dir)
project_path = current_dir[:last_position]
sys.path.append(project_path)

import Handlers as hd

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns

from sklearn.model_selection import train_test_split # сплитование выборки
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from category_encoders import OneHotEncoder, BinaryEncoder

from statsmodels.tsa.stattools import adfuller

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

from sklearn import set_config
set_config(transform_output='pandas')

%matplotlib inline
plt.style.use('seaborn-darkgrid')
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (10, 8)

C:\Users\avvoy\AppData\Local\Temp\ipykernel_18216\1955050319.py:29: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


# Load Data

In [3]:
data = pd.read_parquet('..\ETH-Full-1H.parquet')

In [4]:
df = data.copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49337 entries, 2017-09-01 00:00:00 to 2023-04-23 23:00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    49337 non-null  float64
 1   High    49337 non-null  float64
 2   Low     49337 non-null  float64
 3   Close   49337 non-null  float64
 4   Volume  49337 non-null  float64
 5   Trades  49337 non-null  int32  
dtypes: float64(5), int32(1)
memory usage: 2.4 MB


In [6]:
df.index

DatetimeIndex(['2017-09-01 00:00:00', '2017-09-01 01:00:00',
               '2017-09-01 02:00:00', '2017-09-01 03:00:00',
               '2017-09-01 04:00:00', '2017-09-01 05:00:00',
               '2017-09-01 06:00:00', '2017-09-01 07:00:00',
               '2017-09-01 08:00:00', '2017-09-01 09:00:00',
               ...
               '2023-04-23 14:00:00', '2023-04-23 15:00:00',
               '2023-04-23 16:00:00', '2023-04-23 17:00:00',
               '2023-04-23 18:00:00', '2023-04-23 19:00:00',
               '2023-04-23 20:00:00', '2023-04-23 21:00:00',
               '2023-04-23 22:00:00', '2023-04-23 23:00:00'],
              dtype='datetime64[ns]', name='Datetime', length=49337, freq=None)

Понятно что индексы часовые, но частота не высталена

In [8]:
# df = df.asfreq('H') # этот вариант сработает, но на месте пропусков появятся NaN записи

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49337 entries, 2017-09-01 00:00:00 to 2023-04-23 23:00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    49337 non-null  float64
 1   High    49337 non-null  float64
 2   Low     49337 non-null  float64
 3   Close   49337 non-null  float64
 4   Volume  49337 non-null  float64
 5   Trades  49337 non-null  int32  
dtypes: float64(5), int32(1)
memory usage: 2.4 MB


# Feature Enginiring

In [10]:
df

,Open,High,Low,Close,Volume,Trades
Datetime,,,,,,
2017-09-01 00:00:00,384.79,388.74,384.79,388.74,162.97779,131
2017-09-01 01:00:00,388.69,394.06,387.53,391.37,248.72949,262
2017-09-01 02:00:00,392.88,394.39,389.46,390.88,198.82856,225
2017-09-01 03:00:00,391.70,393.13,390.86,390.86,141.56114,138
2017-09-01 04:00:00,392.65,394.27,387.64,390.33,135.65894,128
...,...,...,...,...,...,...
2023-04-23 19:00:00,1846.76,1852.35,1844.92,1846.49,16550.18290,21648
2023-04-23 20:00:00,1846.49,1852.30,1838.03,1848.89,13113.47550,19445
2023-04-23 21:00:00,1848.88,1859.32,1847.59,1854.66,7855.61330,13824


In [11]:
date_time = df.index

In [7]:
def add_datetime_features(df, as_category=True):
    """Функция для генерации признаков даты/времени из индекса датафрейма

    Args:
        df (_type_): исходный датафрейм
        as_category (bool, optional): Кодидровать ли созданные признаки. Defaults to True.

    Returns:
        _type_: возвращает новый датафрейм
    """
    
    new_df = df.copy()
    
    date_time = new_df.index
    new_df['M'] = date_time.month # Month
    new_df['W'] = date_time.isocalendar().week # Week from year begin
    new_df['MD'] = date_time.day # Day of Month
    new_df['WD'] = date_time.weekday # Day of Week
    new_df['H'] = date_time.hour # Hour
    
    if as_category:
        new_df['M'] = new_df['M'].astype('category')
        new_df['W'] = new_df['W'].astype('category')
        new_df['MD'] = new_df['MD'].astype('category')
        new_df['WD'] = new_df['WD'].astype('category')
        new_df['H'] = new_df['H'].astype('category')
        
    return new_df

In [8]:
df = add_datetime_features(df)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49337 entries, 2017-09-01 00:00:00 to 2023-04-23 23:00:00
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   Open    49337 non-null  float64 
 1   High    49337 non-null  float64 
 2   Low     49337 non-null  float64 
 3   Close   49337 non-null  float64 
 4   Volume  49337 non-null  float64 
 5   Trades  49337 non-null  int32   
 6   M       49337 non-null  category
 7   W       49337 non-null  category
 8   MD      49337 non-null  category
 9   WD      49337 non-null  category
 10  H       49337 non-null  category
dtypes: category(5), float64(5), int32(1)
memory usage: 2.7 MB


# Обработка признаков

Отбор списка категориальных признаков 

In [10]:
cat_columns = df.select_dtypes(include=['category']).columns.to_list()
cat_columns

['M', 'W', 'MD', 'WD', 'H']

In [11]:
num_columns = df.select_dtypes(include=['int', 'float']).columns.to_list()
num_columns

['Open', 'High', 'Low', 'Close', 'Volume', 'Trades']

Анализ уникальных значений для определения категорий и как их кодировать

In [12]:
hd.get_top_unique_values(df)

,Column Name,Count Unique,% Unique,Top Value,Top Value Count,Top Value %
0,Open,40219,81.52,282.0,10,0.02
1,High,37884,76.79,299.0,15,0.03
2,Low,38124,77.27,460.0,17,0.03
3,Close,40199,81.48,282.0,8,0.02
4,Trades,29653,60.10,241,15,0.03
5,M,12,0.02,1,4463,9.05
6,W,53,0.11,13,1008,2.04
7,MD,31,0.06,1,1632,3.31
8,WD,7,0.01,5,7077,14.34
9,H,24,0.05,0,2060,4.18


W, MD, H нужно кодировать BinaryEncoder, больше 15 категорий, надо кодировать

### OneHotEncoder

In [13]:
cols_to_ohe = ['M', 'WD']
ohe = OneHotEncoder(cols=cols_to_ohe)
df = ohe.fit_transform(df)

### BinaryEncoder

In [14]:
cols_to_be = ['W', 'MD', 'H']
be = BinaryEncoder(cols=cols_to_be)
df = be.fit_transform(df)

In [221]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49337 entries, 2017-09-01 00:00:00 to 2023-04-23 23:00:00
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    49337 non-null  float64
 1   High    49337 non-null  float64
 2   Low     49337 non-null  float64
 3   Close   49337 non-null  float64
 4   Volume  49337 non-null  float64
 5   Trades  49337 non-null  int32  
 6   M_1     49337 non-null  int64  
 7   M_2     49337 non-null  int64  
 8   M_3     49337 non-null  int64  
 9   M_4     49337 non-null  int64  
 10  M_5     49337 non-null  int64  
 11  M_6     49337 non-null  int64  
 12  M_7     49337 non-null  int64  
 13  M_8     49337 non-null  int64  
 14  M_9     49337 non-null  int64  
 15  M_10    49337 non-null  int64  
 16  M_11    49337 non-null  int64  
 17  M_12    49337 non-null  int64  
 18  W_0     49337 non-null  int64  
 19  W_1     49337 non-null  int64  
 20  W_2     49337 non-null  int64  
 21  

### Scaler
Нужно использовать после разделения выборки, fit только на тренировочной выборке

In [222]:
# Columns to be scaled
cols_to_scale = ['Open', 'High', 'Low', 'Close', 'Volume', 'Trades']

# Initialize a scaler
scaler = StandardScaler()

# Apply scaler to the columns to be scaled
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [223]:
df

,Open,High,Low,Close,Volume,Trades,M_1,M_2,M_3,M_4,...,WD_3,WD_4,WD_5,WD_6,WD_7,H_0,H_1,H_2,H_3,H_4
Datetime,,,,,,,,,,,,,,,,,,,,,
2017-09-01 00:00:00,-0.646609,-0.645790,-0.643818,-0.643201,-0.838597,-0.765864,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2017-09-01 01:00:00,-0.643224,-0.641202,-0.641423,-0.640918,-0.835589,-0.761485,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2017-09-01 02:00:00,-0.639587,-0.640917,-0.639737,-0.641343,-0.837339,-0.762722,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2017-09-01 03:00:00,-0.640611,-0.642004,-0.638513,-0.641361,-0.839348,-0.765630,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2017-09-01 04:00:00,-0.639787,-0.641021,-0.641327,-0.641821,-0.839555,-0.765964,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-23 19:00:00,0.622284,0.616534,0.632181,0.622027,-0.263759,-0.046546,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0
2023-04-23 20:00:00,0.622050,0.616490,0.626160,0.624110,-0.384314,-0.120193,0,0,0,0,...,1,0,0,0,0,1,0,1,0,1
2023-04-23 21:00:00,0.624124,0.622545,0.634514,0.629118,-0.568751,-0.308104,0,0,0,0,...,1,0,0,0,0,1,0,1,1,0


In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49337 entries, 2017-09-01 00:00:00 to 2023-04-23 23:00:00
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    49337 non-null  float64
 1   High    49337 non-null  float64
 2   Low     49337 non-null  float64
 3   Close   49337 non-null  float64
 4   Volume  49337 non-null  float64
 5   Trades  49337 non-null  float64
 6   M_1     49337 non-null  int64  
 7   M_2     49337 non-null  int64  
 8   M_3     49337 non-null  int64  
 9   M_4     49337 non-null  int64  
 10  M_5     49337 non-null  int64  
 11  M_6     49337 non-null  int64  
 12  M_7     49337 non-null  int64  
 13  M_8     49337 non-null  int64  
 14  M_9     49337 non-null  int64  
 15  M_10    49337 non-null  int64  
 16  M_11    49337 non-null  int64  
 17  M_12    49337 non-null  int64  
 18  W_0     49337 non-null  int64  
 19  W_1     49337 non-null  int64  
 20  W_2     49337 non-null  int64  
 21  